In [1]:
import cv2
import numpy as np
import napari

In [2]:

# Read all images
images = [cv2.imread(f"Metsys Data/44/Acquire_0/25/0/{i}.bmp",cv2.IMREAD_GRAYSCALE) for i in range(20)]

# Stack all images vertically
stacked_image = np.array(images)


viewer = napari.Viewer()
viewer.add_image(stacked_image)

<Image layer 'stacked_image' at 0x21461224100>

### FFT for registration
* Image Loading
* Laplacian Variance Function:
    * Purpose: To evaluate the sharpness or focus measure of an image
* Computing Relative Shifts:
    * Computes 2D Fast Fourier Transforms (FFTs) of two consecutive images.
    * Calculates the normalized cross-power spectrum
    * Detects the shift by identifying the position of maximum correlation in the spatial domain.
    * Adjusts the detected shift to handle the FFT's wraparound.

In [3]:
images = [cv2.imread(f"Metsys Data/44/Acquire_0/25/0/{i}.bmp",cv2.IMREAD_GRAYSCALE) for i in range(20)]


def laplacian_variance(image):
    return cv2.Laplacian(image, cv2.CV_64F).var()

# Compute the relative shifts
shifts = [(0, 0)]
for i in range(1, len(images)):
    dft_A = np.fft.fft2(images[i-1])
    dft_B = np.fft.fft2(images[i])
    
    cross_power = dft_A * np.conj(dft_B)
    R = cross_power / np.abs(cross_power)
    
    shift = np.unravel_index(np.argmax(np.fft.ifft2(R).real), images[i].shape)
    
    # Convert to relative shift (taking care of the wraparound from FFT)
    shift = (shift[0] if shift[0] <= images[i].shape[0] // 2 else shift[0] - images[i].shape[0],
             shift[1] if shift[1] <= images[i].shape[1] // 2 else shift[1] - images[i].shape[1])
    
    shifts.append(shift)

max_shift_x = int(max([shift[1] for shift in shifts]))
max_shift_y = int(max([shift[0] for shift in shifts]))

panorama_width = images[0].shape[1] + max_shift_x * len(images)
panorama_height = images[0].shape[0] + max_shift_y * len(images)



### Stack
* make an stack by shifts list

In [4]:
plane_image = np.zeros((20, panorama_height, panorama_width))

z = 0
y = 0 

images = np.array(images)

for img, (y_shift, _) in zip(images, shifts):
    size_y, size_x = img.shape
    y += y_shift
    y_start = y
    y_end = y + size_y
    plane_image[z, y_start: y_end, :] = images[z]
    
    z += 1  # Increment z for the next image


### take the shift_m

In [6]:
shift_s = 0
for i in range(20):
    shift_s += shifts[i][0]
shift_m = int(shift_s/19)

In [12]:
plane_image_shift_m = np.zeros((20, panorama_height, panorama_width))

z = 0
y = 0 

images = np.array(images)

for img in images:
    size_y, size_x = img.shape
    if z==0:
        y=0
    else:
        y += shift_m
    y_start = y
    y_end = y + size_y
    plane_image_shift_m[z, y_start: y_end, :] = images[z]
    z = z+1

In [6]:
viewer = napari.Viewer()
viewer.add_image(plane_image.astype('uint8'))
#viewer.add_image(plane_image_shift_m)

<Image layer 'Image' at 0x214009d8220>

### Alpha Blending

In [ ]:
# Initialize the canvas
canvas = np.zeros((panorama_height, panorama_width), dtype=np.uint8)

# Starting position for placing images
x_offset = 0
y_offset = 0

for i, img in enumerate(images):
    # Get the shift offset
    y_offset += shifts[i][0]
    if i != 0:
        overlap_start = y_offset    
        overlap_end = y_offset + img.shape[0] - shifts[i][0]# No overlap for the first image
        for y in range(img.shape[0]):
            for x in range(img.shape[1]):
                canvas_y = y + y_offset
                if overlap_start <= canvas_y <= overlap_end and canvas[canvas_y, x] != 0:
                    # Calculate the weight for blending
                    alpha = (canvas_y - overlap_start) / (overlap_end - overlap_start) # 0 to 1
                    canvas[canvas_y, x] = (1-alpha) * canvas[canvas_y, x] + (alpha) * img[y, x]
                else:
                        canvas[canvas_y, x + x_offset] = img[y, x]
    else:
        # For the first image
        canvas[0: img.shape[0], :] = images[0]

plane_image_alpha = canvas


### time efficiency

In [5]:
canvas = np.zeros((panorama_height, panorama_width), dtype=np.uint8)


x_offset = 0 
y_offset = 0

for i, img in enumerate(images):
    # Determine overlap regions
    overlap_start = y_offset + shifts[i][0]
    overlap_end = y_offset + img.shape[0]

    overlap = canvas[overlap_start:overlap_end, :]


    if i != 0 and np.any(overlap):
        # create an alpha mask
        mask = np.linspace(1, 0, overlap.shape[0])[np.newaxis, :].T
        mask = np.repeat(mask, overlap.shape[1], axis=1)
        
        # Apply the mask for blending
        foreground = cv2.multiply(mask, overlap.astype(float))
        background = cv2.multiply(1.0 - mask, img[:overlap.shape[0]].astype(float))

        blended = cv2.add(foreground, background).astype(np.uint8)

        canvas[overlap_start:overlap_end, :] = blended
        canvas[overlap_end:overlap_end + shifts[i][0], :] = img[overlap.shape[0]:, :]
    else:
        # for first image
        canvas[0:overlap_end, :] = img
    
    y_offset += shifts[i][0]

plane_image_alpha2 = canvas

In [24]:
viewer = napari.Viewer()
viewer.add_image(stacked_image)
viewer.add_image(plane_image.astype('uint8'))
#viewer.add_image(plane_image_alpha)
viewer.add_image(plane_image_alpha2)


<Image layer 'plane_image_alpha2' at 0x1ca57ab6640>

### Multiband Blending

* Build Laplacian pyramids $L1$ and $L2$ from images 1 and 2
* Build a Gaussian pyramid $GM$ from selection mask M
* Form a combined pyramid $LS$ from $L1$ and $L2$ using $GM$ as weights:
    * $LS = GM * L1 + (1-GM) * L2$
* Collapse the $LS$ pyramid to get the final blended image

In [6]:
# Create Gaussian and Laplacian pyramids
def build_gaussian_pyramid(img, levels):
    pyramid = [img]
    for i in range(levels - 1):
        img = cv2.pyrDown(img)
        pyramid.append(img)
    return pyramid

def build_laplacian_pyramid(img, levels):
    gaussian_pyramid = build_gaussian_pyramid(img, levels)
    laplacian_pyramid = []
    for i in range(levels - 1):
        size = (gaussian_pyramid[i].shape[1], gaussian_pyramid[i].shape[0])
        expanded = cv2.pyrUp(gaussian_pyramid[i + 1], dstsize=size)
        laplacian = cv2.subtract(gaussian_pyramid[i], expanded)
        laplacian_pyramid.append(laplacian)
    laplacian_pyramid.append(gaussian_pyramid[-1])  # Add the smallest level as it is
    return laplacian_pyramid
    
# Blend two images using Gaussian pyramids
def blend_images(img1, img2, mask, levels):
    # Generate pyramids
    LP_img1 = build_laplacian_pyramid(img1, levels)
    LP_img2 = build_laplacian_pyramid(img2, levels)
    GP_mask = build_gaussian_pyramid(mask, levels)

    # Blend
    blended_pyramid = []
    for l_img1, l_img2, g_mask in zip(LP_img1, LP_img2, GP_mask):
        blended_pyramid.append(l_img1 * g_mask + l_img2 * (1.0 - g_mask))
    
    # Reconstruct
    reconstructed_image = blended_pyramid[-1]
    for i in range(len(blended_pyramid)-2, -1, -1):
        size = (blended_pyramid[i].shape[1], blended_pyramid[i].shape[0])
        upsampled = cv2.pyrUp(reconstructed_image, dstsize=size)
        reconstructed_image = cv2.add(upsampled, blended_pyramid[i])

    return reconstructed_image

In [7]:
canvas = np.zeros((panorama_height, panorama_width), dtype=np.uint8)

#set pyramid level
level = 2

x_offset = 0  # Assuming vertical stacking of images
y_offset = 0

for i, img in enumerate(images):
    # Determine overlap regions
    overlap_start = y_offset + shifts[i][0]
    overlap_end = y_offset + img.shape[0]

    overlap = canvas[overlap_start:overlap_end, :]


    if i != 0 and np.any(overlap):
        mask = np.ones(overlap.shape)

        # Only blending the overlap region of the canvas (which includes the previous image) and the current image.
        blended = blend_images(overlap, img[0:img.shape[0]-shifts[i][0]], mask, level)
       
        canvas[overlap_start:overlap_end, :] = blended
        canvas[overlap_end:overlap_end + shifts[i][0], :] = img[overlap.shape[0]:, :]
    else:
        # for first image
        canvas[0:overlap_end, :] = img
    
    y_offset += shifts[i][0]

    plane_image_MBB = canvas


In [8]:
viewer = napari.Viewer()
viewer.add_image(plane_image.astype('uint8'))
viewer.add_image(plane_image_alpha2)
viewer.add_image(plane_image_MBB)
viewer.add_image(stacked_image)

<Image layer 'stacked_image' at 0x217a3d6ef70>

### Poisson Blending

In [4]:
def poisson_blend(source, target, mask, offset):
    # Compute the region to be blended
    y_min, y_max, x_min, x_max = compute_bounding_box(mask, offset)

    # Extract regions from source, target, and mask images
    source_region = source[y_min:y_max, x_min:x_max]
    target_region = target[y_min+offset[1]:y_max+offset[1], x_min+offset[0]:x_max+offset[0]]
    mask_region = mask[y_min:y_max, x_min:x_max]

    # Compute the Laplacian of the source region
    laplacian = cv2.Laplacian(source_region, cv2.CV_64F)

    # Here, you'd typically solve the Poisson equation to get the blended region.
    # This is a simplified approach:
    for y in range(mask_region.shape[0]):
        for x in range(mask_region.shape[1]):
            if mask_region[y, x] == 255:  # Pixel is in the mask
                target_region[y, x] += laplacian[y, x]

    result = target.copy()
    result[y_min+offset[1]:y_max+offset[1], x_min+offset[0]:x_max+offset[0]] = target_region
    return result

def compute_bounding_box(mask, offset):
    y_indices, x_indices = np.where(mask == 255)
    y_min = np.min(y_indices)
    y_max = np.max(y_indices) + 1
    x_min = np.min(x_indices)
    x_max = np.max(x_indices) + 1
    return y_min, y_max, x_min, x_max


In [13]:
canvas = np.zeros((panorama_height, panorama_width), dtype=np.uint8)

x_offset = 0 
y_offset = 0

for i, img in enumerate(images):
    # Determine overlap regions
    overlap_start = y_offset + shifts[i][0]
    overlap_end = y_offset + img.shape[0]

    overlap = canvas[overlap_start:overlap_end, :]
    
    if i != 0:  # Not the first image
        # Create a binary mask for Poisson blending. Assuming vertical blend, so overlap will be full width.
        mask = np.zeros_like(img, dtype=np.uint8)
        mask[:overlap_end - overlap_start, :] = 255  # White region for blending

        # Use Poisson blending
        blended = poisson_blend(img, canvas, mask, (x_offset, overlap_start))
        canvas[overlap_start:overlap_end, :] = blended[overlap_start:overlap_end, :]
        canvas[overlap_end:overlap_end + shifts[i][0], :] = img[overlap.shape[0]:, :]
    else:
        # For the first image, just place it on the canvas.
        canvas[0:overlap_end, :] = img
    
    y_offset += shifts[i][0]

plane_image_poisson = canvas

In [33]:
x_offset = 0 
y_offset = 0

for i, img in enumerate(images):
    # Determine overlap regions
    overlap_start = y_offset + shifts[i][0]
    overlap_end = y_offset + img.shape[0]
    if i != 0:
    # Create the mask
        mask = np.zeros(img.shape, dtype=np.uint8)
        mask[:overlap_end - overlap_start, :] = 255

        # Determine center of the overlapping region for seamless cloning
        center_x = int(img.shape[1] // 2)
        center_y = int((overlap_start + overlap_end) // 2)
        center = (center_x, center_y + y_offset)
        print("img min-max:", np.min(img), np.max(img))
        print("canvas min-max:", np.min(canvas), np.max(canvas))
        print("mask min-max:", np.min(mask), np.max(mask))

    # Use cv2.seamlessClone
        canvas = cv2.seamlessClone(img, canvas, mask, center, cv2.NORMAL_CLONE)
    else:
        # For the first image, just place it on the canvas.
        canvas[0:overlap_end, :] = img
    
    y_offset += shifts[i][0]



img min-max: 13 255
canvas min-max: 0 255
mask min-max: 0 255


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\deriv.cpp:792: error: (-215:Assertion failed) !_src.empty() in function 'cv::Laplacian'


In [34]:
dummy_img = np.ones((1920, 2560), dtype=np.uint8) * 255
dummy_canvas = np.zeros((4380, 2560), dtype=np.uint8)
dummy_mask = np.ones((1920, 2560), dtype=np.uint8) * 255
dummy_center = (1280, 1019)

try:
    result = cv2.seamlessClone(dummy_img, dummy_canvas, dummy_mask, dummy_center, cv2.NORMAL_CLONE)
    cv2.imshow("Result", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
except Exception as e:
    print(str(e))

OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\deriv.cpp:792: error: (-215:Assertion failed) !_src.empty() in function 'cv::Laplacian'



In [30]:
overlap_end - overlap_start

1920

In [11]:
viewer = napari.Viewer()
#viewer.add_image(plane_image.astype('uint8'))
viewer.add_image(plane_image_poisson)


<Image layer 'plane_image_poisson' at 0x1ca0eebab80>

### TEST

In [ ]:
from PoissonBlending import blend

canvas = np.zeros((panorama_height, panorama_width), dtype=np.uint8)

x_offset = 0 
y_offset = 0

for i, img in enumerate(images):
    # Determine overlap regions
    overlap_start = y_offset + shifts[i][0]
    overlap_end = y_offset + img.shape[0]

    if i != 0:
        # Determine the mask for Poisson blending
        mask = np.zeros_like(img, dtype=np.uint8)
        mask[:overlap.shape[0], :] = 255  # this mask will be used to blend the overlapping region

        # Apply Poisson blending
        offset = (0, overlap_start)
        canvas = blend(canvas, img, mask, offset)
    else:
        # For the first image
        canvas[0:overlap_end, :] = img

    y_offset += shifts[i][0]

plane_image_alpha2 = canvas


In [84]:
for i, img in enumerate(images):
    # Determine overlap regions
    overlap_start = y_offset + shifts[i][0]
    overlap_end = y_offset + img.shape[0]
    overlap = canvas[overlap_start:overlap_end, :]
    print(overlap.shape)
    y_offset += shifts[i][0]

(1920, 2560)
(1801, 2560)
(1800, 2560)
(1797, 2560)
(1797, 2560)
(1798, 2560)
(1799, 2560)
(1800, 2560)
(1798, 2560)
(1800, 2560)
(1799, 2560)
(1800, 2560)
(1800, 2560)
(1798, 2560)
(1798, 2560)
(1800, 2560)
(1800, 2560)
(1800, 2560)
(1799, 2560)
(1802, 2560)


In [85]:
mask = np.ones(overlap.shape)

In [86]:
canvas = np.zeros((panorama_height, panorama_width), dtype=np.uint8)
img = images[0]

canvas[0:1920,:]=img


In [87]:
i = 1
img1 = images[0]
img = images[i]

y_offset = 0

overlap_start = y_offset + shifts[i][0]
overlap_end = y_offset + img.shape[0]
overlap = canvas[overlap_start:overlap_end, :]


if i != 0 and np.any(overlap):
        mask = np.ones(overlap.shape)
        print(mask.shape)
        

(1801, 2560)


In [37]:
blended_pyramid = []
for l_img1, l_img2, g_mask in zip(LP_img1, LP_img2, GP_mask):
    print(l_img1.shape,l_img2.shape,g_mask.shape)
    blended_pyramid.append(l_img1 * g_mask + l_img2 * (1.0 - g_mask))

(1801, 2560) (1801, 2560) (1801, 2560)
(901, 1280) (901, 1280) (901, 1280)
(451, 640) (451, 640) (451, 640)
(226, 320) (226, 320) (226, 320)


In [39]:
reconstructed_image = blended_pyramid[-1]
for i in range(len(blended_pyramid)-2, -1, -1):
    size = (blended_pyramid[i].shape[1], blended_pyramid[i].shape[0])
    upsampled = cv2.pyrUp(reconstructed_image, dstsize=size)
    reconstructed_image = cv2.add(upsampled, blended_pyramid[i])

In [41]:
viewer = napari.Viewer()
viewer.add_image(overlap)
viewer.add_image(img1)
viewer.add_image(reconstructed_image)

<Image layer 'reconstructed_image' at 0x27900960250>